In [111]:
# Primeiro passo: baixar boletinss de urna aqui: https://dados.gov.br/dataset/resultados-2022-boletim-de-urna 
# Detalhe importante: todos os arquivos são datados de 31.10.2022. O papo de que eles foram "alterados" depois é fake news.
# Abrir os zips, e jogar os .csv no mesmo diretório deste notebook
import os

boletins_ufs = [x for x in os.listdir() if x.startswith("bweb_2t")]

In [263]:
boletins_ufs

['bweb_2t_PB_311020221535.csv',
 'bweb_2t_MG_311020221535.csv',
 'bweb_2t_MS_311020221535.csv',
 'bweb_2t_PI_311020221535.csv',
 'bweb_2t_AP_311020221535.csv',
 'bweb_2t_SC_311020221535.csv',
 'bweb_2t_CE_311020221535.csv',
 'bweb_2t_BA_311020221535.csv',
 'bweb_2t_PR_311020221535.csv',
 'bweb_2t_RS_311020221535.csv',
 'bweb_2t_GO_311020221535.csv',
 'bweb_2t_ZZ_311020221535.csv',
 'bweb_2t_RR_311020221535.csv',
 'bweb_2t_SE_311020221535.csv',
 'bweb_2t_TO_311020221535.csv',
 'bweb_2t_RJ_311020221535.csv',
 'bweb_2t_DF_311020221535.csv',
 'bweb_2t_AM_311020221535.csv',
 'bweb_2t_RN_311020221535.csv',
 'bweb_2t_MA_311020221535.csv',
 'bweb_2t_ES_311020221535.csv',
 'bweb_2t_SP_311020221535.csv',
 'bweb_2t_AC_311020221535.csv',
 'bweb_2t_PA_311020221535.csv',
 'bweb_2t_PE_311020221535.csv',
 'bweb_2t_AL_311020221535.csv',
 'bweb_2t_RO_311020221535.csv',
 'bweb_2t_MT_311020221535.csv']

In [170]:
import pandas as pd

# Função que pega csv, transforma em dataframe, e retorna seções onde um candidato teve 100% dos votos
def extrair_outliers(csv): 
    df = pd.read_csv(csv, sep=";", encoding="latin-1")
    df['zona_secao'] = df['NR_ZONA'].astype(str) + df['NR_SECAO'].astype(str)
    municipios = df['NM_MUNICIPIO'].unique()
    outliers = []
    for municipio in municipios:
        m = df.query(f'NM_MUNICIPIO == "{municipio}" & DS_ELEICAO == "Eleição Geral Federal 2022"').copy()
        for zona_secao in m['zona_secao'].unique():
            s = m.query(f'zona_secao == "{zona_secao}"')
            if 'LULA' not in s['NM_VOTAVEL'].values or 'JAIR BOLSONARO' not in s['NM_VOTAVEL'].values:
                data = s.to_dict("records")
                outliers += data
    return outliers
    

In [171]:
# Colocamos todos os outliers em uma mesma lista de dicionários. Pode levar muitos minutos para rodar essa parte
# dependendo da sua máquina
outliers = []
for csv in sorted(boletins_ufs):
    outliers += extrair_outliers(csv)

In [175]:
# Criamos um DataFrame com todos esses dados
df = pd.DataFrame(outliers)

In [216]:
# Agora damos uma limpada no dataframe, tirando algumas colunas, incluindo outras 
out = []
for secao in df['zona_secao'].unique():
    s = df.query(f"zona_secao == '{secao}'").copy()
    colunas = ['SG_UF', 'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'QT_APTOS', 'QT_ABSTENCOES']
    obj = s.groupby("NM_VOTAVEL")['QT_VOTOS'].sum().to_dict()
    if 'LULA' not in obj.keys():
        obj['LULA'] = 0
    elif 'JAIR BOLSONARO' not in obj.keys():
        obj['JAIR BOLSONARO'] = 0
    for c in colunas:
        obj[c] = s[c].values[0]
        out.append(obj)
        

In [257]:
# Por últimos acrescentamos dados de locais de votação, daqui: https://dados.gov.br/dataset/eleitorado-2022
locais = pd.read_csv("/Users/burgos/Downloads/eleitorado_local_votacao_2022 (1)/eleitorado_local_votacao_2022.csv", sep=";", encoding="latin-1")

def obter_dados_secao(data_point):
    municipio = data_point['NM_MUNICIPIO']
    zona = data_point['NR_ZONA'] 
    secao = data_point['NR_SECAO']
    data = locais.query(f"NM_MUNICIPIO == '{municipio}' & NR_ZONA == {zona} & NR_SECAO == {secao}")
    return data[['NM_LOCAL_VOTACAO', 'NM_BAIRRO', 'NR_CEP', 'NR_TELEFONE_LOCAL']].to_dict('records')[1]
    
    

In [259]:
# Acrescentamos os locais das seções à lista de dicionários principal
for o in out:
    o.update(obter_dados_secao(o))

In [262]:
# E finalmente exportamos tudo
pd.DataFrame(out).drop_duplicates().to_csv("new_outliers.csv")